In [47]:
import pandas as pd
import numpy as np
import random
import os
import glob
import shutil
import sqlite3
from tqdm import tqdm
from typing import List

In [48]:
sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))

In [49]:
def generation_datasets(number_datasets:int, 
                        number_rows:int,
                        path_data:str="/content/data/")->None:
  if os.path.exists(path_data):
    shutil.rmtree(path_data)
  if not os.path.exists(path_data):
    os.makedirs(path_data)
  for n in tqdm(range(number_datasets)):
    val = int("".join([str(10),str(n)]))
    list_code = [val for _ in range(number_rows)]
    list_val = [random.randint(1, 100) for _ in range(number_rows)]
    d = {"code":list_code, "val":list_val}
    df = pd.DataFrame(data=d)
    df.to_csv(f"{path_data}dataset_{n}.csv", index=False, sep=',')

In [50]:
generation_datasets(3,1000000)

100%|██████████| 3/3 [00:08<00:00,  2.98s/it]


In [51]:
def create_table(name_table:str="test_data", 
                 col1:str ="code", 
                 col2:str="val",
                 path_database:str="/content/test.db") -> None:
    """Создать таблицу в БД"""
    try:
        conn = sqlite3.connect(f"{path_database}")
        cursor = conn.cursor()
        cursor.execute(f"drop table if exists {name_table};")
        cursor.execute(f"create table {name_table}({col1} integer, {col2} integer);")
        conn.commit()
    except sqlite3.DatabaseError as err:
        print("Ошибка при работе с SQLite")
    finally:
        if conn:
            cursor.close()
            conn.close()
            print("Соединение с SQlite закрыто, таблица сформирована")

In [52]:
create_table()

Соединение с SQlite закрыто, таблица сформирована


In [53]:
def get_all_file(path_data: str = '/content/data/') -> List[str]:
    """Получить список файлов в папке"""
    all_file_name = list(glob.glob(f'{path_data}*.csv', recursive=True))
    return all_file_name

In [54]:
def load_data(path_database:str="/content/test.db",
              table_name:str = 'test_data',
              col1:str ="code", 
              col2:str="val") -> None:
    """Записать данные в БД"""
    try:
        conn = sqlite3.connect(path_database)
        cursor = conn.cursor()
        for current_file in get_all_file():
          query = f"INSERT INTO {table_name} ({col1}, {col2}) values(?, ?);"
          data = list(pd.read_csv(current_file).to_records(index=False))
          cursor.executemany(query, data)
          conn.commit()
    except sqlite3.DatabaseError as err:
        print("Ошибка при работе с SQLite")
    finally:
        if conn:
            cursor.close()
            conn.close()
            print("Соединение с SQlite закрыто, данные добавлены")

In [55]:
%%time
load_data()

Соединение с SQlite закрыто, данные добавлены
CPU times: user 19.3 s, sys: 276 ms, total: 19.6 s
Wall time: 19.9 s


In [56]:
def select_sqlite(sql, path_database:str="/content/test.db")->pd.DataFrame:
    conn = sqlite3.connect(path_database)
    return pd.read_sql(sql, conn)

In [57]:
sql = """select t.code, count(*) as total_amount from test_data as t group by t.code"""

In [58]:
select_sqlite(sql)

,code,total_amount
0,100,1000000
1,101,1000000
2,102,1000000
